In [1]:
import json
import pandas as pd

In [4]:
with open('../extraction/es_convs.txt', 'r') as f:
    convs = pd.read_json(f, orient='table')
    
with open('../extraction/es_users.txt', 'r') as f:
    users = pd.read_json(f, orient='table')
    
display(convs.head())
display(users.head())

,in_reply_to_user_id,conversation_id,author_id,lang,text,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
0,69416519,1402624093946527745,1537353780,es,"Muchas fuerza Florencia,lo que te desea el mal...",1402658383950454791,0,0,0,0
1,69416519,1402624093946527745,468895557,es,El mal que hace la madre lo pagan los hijos.\n...,1402656218150285315,0,0,0,0
2,69416519,1402624093946527745,1471308854,es,Nunca un hospital público estos garcas. Ni ver...,1402655563792764935,0,0,0,0
3,69416519,1402624093946527745,398461357,es,Porque no muestra la cara ?,1402653558219194372,0,0,0,0
4,69416519,1402624093946527745,169167606,es,Esta mujer esta anoréxica,1402653194174468103,0,0,1,0


,id,name,username,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,1537353780,vicky Sanchez,vicky_sanchez75,30,276,3744,0
1,722604305224724481,Matapanopticus,matapanopticus,71,503,1627,0
2,468895557,GracielaGonzalez,Graciela_Gonz,953,1264,50746,5
3,1471308854,Adrian espindola,AdrEspindola,19,236,252,0
4,157010026,Carey Mahoney,CareyMahoney34,174,1710,3762,2


In [5]:
# Rename the users 'id' > 'author_id' to join convo-users
users.rename(columns={'id':'author_id'}, inplace=True)

In [6]:
data = convs.merge(users, how='left', on='author_id')
data.columns

Index(['in_reply_to_user_id', 'conversation_id', 'author_id', 'lang', 'text',
       'id', 'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count', 'name',
       'username', 'public_metrics.followers_count',
       'public_metrics.following_count', 'public_metrics.tweet_count',
       'public_metrics.listed_count'],
      dtype='object')

### Cleaning

In [7]:
data.rename(columns=lambda x: x.replace('public_metrics.', ''), inplace=True)

In [8]:
data.columns

Index(['in_reply_to_user_id', 'conversation_id', 'author_id', 'lang', 'text',
       'id', 'retweet_count', 'reply_count', 'like_count', 'quote_count',
       'name', 'username', 'followers_count', 'following_count', 'tweet_count',
       'listed_count'],
      dtype='object')

In [9]:
data.drop(columns=['lang', 'listed_count', 'name', 'quote_count'], inplace=True)

In [10]:
col_map = {'in_reply_to_user_id':'reply_id', 
           'retweet_count':'retweets', 
           'reply_count':'replies', 
           'like_count':'likes', 
           'tweet_count':'tweets'}

In [11]:
data.rename(columns=col_map, inplace=True)

In [12]:
data.head()

,reply_id,conversation_id,author_id,text,id,retweets,replies,likes,username,followers_count,following_count,tweets
0,69416519,1402624093946527745,1537353780,"Muchas fuerza Florencia,lo que te desea el mal...",1402658383950454791,0,0,0,vicky_sanchez75,30,276,3744
1,69416519,1402624093946527745,468895557,El mal que hace la madre lo pagan los hijos.\n...,1402656218150285315,0,0,0,Graciela_Gonz,953,1264,50746
2,69416519,1402624093946527745,1471308854,Nunca un hospital público estos garcas. Ni ver...,1402655563792764935,0,0,0,AdrEspindola,19,236,252
3,69416519,1402624093946527745,398461357,Porque no muestra la cara ?,1402653558219194372,0,0,0,madelaguila,160,222,37375
4,69416519,1402624093946527745,169167606,Esta mujer esta anoréxica,1402653194174468103,0,0,1,jdgarzulo,6584,6173,125134


In [13]:
c = len(data.loc[:, 'conversation_id'].unique())
u = len(data.loc[:, 'author_id'].unique())

print(f'EN:\nUnique convos: {c}, unique users: {u}')

EN:
Unique convos: 8, unique users: 87


In [14]:
# Removing duplicated text
data.drop(data.loc[data.loc[:, 'text'].duplicated()].index, inplace=True)

In [15]:
# Pattern to identify mentions
mentions_pat = r'@[\w]+'

data.loc[:, 'text'] = data.loc[:, 'text']\
    .str.replace(mentions_pat, '', regex=True)

In [16]:
with open('es_cleaned.txt', 'w') as d:
    d.writelines(data.to_json(orient='table', force_ascii=False))